In [38]:
import requests
from datetime import datetime
from bs4 import BeautifulSoup
from functools import reduce
import operator

In [83]:
def get_media_list():
    response = requests.get('https://m.news.naver.com/newspaper/home.nhn')
    soup = BeautifulSoup(response.text, 'html.parser')
    return [{'id': media.get('id'), 'title': media.find('img')['alt']} for media in soup.find('ul', {'class': ['offc_lst', '_headline_list']}).find_all('li')]

def get_article_list(media, ymd):
    url = 'https://media.naver.com/press/{}/newspaper?date={}'
    response = requests.get(url.format(media, ymd))
    soup = BeautifulSoup(response.text, 'html.parser')
    article_list = [page.find_all('li') for page in soup.find_all('ul', {'class': 'newspaper_article_lst'})]
    if article_list:
        article_list = reduce(operator.concat, article_list)
    article_list = [{'title': article.find('strong').text, 'link': article.find('a')['href']} for article in article_list]
    return article_list

In [85]:
media_list = get_media_list()
ymd = datetime.now().strftime('%Y%m%d')
for media in media_list:
    article_list = get_article_list(media['id'], ymd)
    print(media['title'], len(article_list))

조선일보 129
전자신문 129
세계일보 90
디지털타임스 98
파이낸셜뉴스 105
헤럴드경제 105
한국일보 78
매일경제 161
한국경제 154
서울신문 117
머니투데이 83
코리아헤럴드 16
서울경제 140
한겨레 78
동아일보 107
국민일보 95
중앙일보 75
경향신문 92
이데일리 86
문화일보 0
아시아경제 0


In [99]:
from IPython.core.display import display, HTML
display(HTML('''
<!DOCTYPE html>
<html>
<head>
  <link href="https://fonts.googleapis.com/css?family=Roboto:100,300,400,500,700,900|Material+Icons" rel="stylesheet">
  <link href="https://cdn.jsdelivr.net/npm/vuetify/dist/vuetify.min.css" rel="stylesheet">
  <meta name="viewport" content="width=device-width, initial-scale=1, maximum-scale=1, user-scalable=no, minimal-ui">
</head>
<body>
  <div id="app">
    <v-app>
      <v-content>
          <v-layout row>
            <v-flex xs12 sm2 offset-sm3>
              <v-card>
                <v-toolbar color="pink" dark>
                  <v-toolbar-title>Inbox</v-toolbar-title>
                </v-toolbar>

                <v-list>
                  <template v-for="(item, index) in items">
                    <v-list-tile
                      :key="item.title"
                      @click="toggle(index)"
                    >
                      <v-list-tile-content>
                        <v-list-tile-title>{{ item.title }}</v-list-tile-title>
                      </v-list-tile-content>
                    </v-list-tile>
                  </template>
                </v-list>
              </v-card>
            </v-flex>
          </v-layout>
      </v-content>
    </v-app>
  </div>

  <script src="https://cdn.jsdelivr.net/npm/vue/dist/vue.js"></script>
  <script src="https://cdn.jsdelivr.net/npm/vuetify/dist/vuetify.js"></script>
  <script>
    new Vue({ 
        el: '#app',
        data: {
            selected: [2],
            items: [
              {
                action: '15 min',
                headline: 'Brunch this weekend?',
                title: 'Ali Connors',
                subtitle: "I'll be in your neighborhood doing errands this weekend. Do you want to hang out?"
              },
              {
                action: '2 hr',
                headline: 'Summer BBQ',
                title: 'me, Scrott, Jennifer',
                subtitle: "Wish I could come, but I'm out of town this weekend."
              },
              {
                action: '6 hr',
                headline: 'Oui oui',
                title: 'Sandra Adams',
                subtitle: 'Do you have Paris recommendations? Have you ever been?'
              },
              {
                action: '12 hr',
                headline: 'Birthday gift',
                title: 'Trevor Hansen',
                subtitle: 'Have any ideas about what we should get Heidi for her birthday?'
              },
              {
                action: '12 hr',
                headline: 'Birthday gift',
                title: 'Trevor Hansen',
                subtitle: 'Have any ideas about what we should get Heidi for her birthday?'
              },
              {
                action: '12 hr',
                headline: 'Birthday gift',
                title: 'Trevor Hansen',
                subtitle: 'Have any ideas about what we should get Heidi for her birthday?'
              },
              {
                action: '12 hr',
                headline: 'Birthday gift',
                title: 'Trevor Hansen',
                subtitle: 'Have any ideas about what we should get Heidi for her birthday?'
              },
              {
                action: '18hr',
                headline: 'Recipe to try',
                title: 'Britta Holt',
                subtitle: 'We should eat this: Grate, Squash, Corn, and tomatillo Tacos.'
              }
            ]        
        },
        methods: {
         toggle (index) {
            const i = this.selected.indexOf(index)

            if (i > -1) {
              this.selected.splice(i, 1)
            } else {
              this.selected.push(index)
            }
          }        
        }
    })
  </script>
</body>
</html>
'''))